In [4]:
titulo = "¿Quien es  \n lady Deadpool?"
titulo_texto = "conoce a "
nombre_descargado = "video16.mp4" 

In [2]:
#Librerias 
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from googletrans import Translator
from TTS.api import TTS
import moviepy.editor as mp
import librosa
import numpy as np
import soundfile as sf
from moviepy.editor import VideoFileClip, AudioFileClip, TextClip, CompositeVideoClip
import moviepy.editor as mpe
import cv2
from PIL import Image
import whisper
from IPython.display import Audio, display
import pyrubberband
import os

ruta_carpeta_principal = "C://Users//R20//Desktop//videostraduccidos//"
ruta_carpeta_audios_esp = "audios_creados//audio_esp_resumen.wav"
ruta_carpeta_audios_ing = "audios_creados//audio_ing_resumen.wav"
ruta_carpeta_audios_fin = "audios_creados//audio_fin_resumen.wav"
ruta_carpeta_videofin = "videos_creados//" + nombre_descargado
ruta_carpeta_videofin_editado = "videos_creados//finales" + nombre_descargado
nombre_video = "videos_descargados//" + nombre_descargado

rutaVideo = ruta_carpeta_principal + nombre_video
ruta_video_final_editado = ruta_carpeta_principal + ruta_carpeta_videofin_editado
rutaAudioIngles = ruta_carpeta_principal + ruta_carpeta_audios_ing
rutaAudioGenerado =  ruta_carpeta_principal + ruta_carpeta_audios_esp
rutaAudioGeneradoNuevaVelocidad = ruta_carpeta_principal + ruta_carpeta_audios_fin
ruta_video_final = ruta_carpeta_principal + ruta_carpeta_videofin
ruta_imagenes = ruta_carpeta_principal + "imagenes//"

In [ ]:
# Import the necessary libraries
import torch
import transformers
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from transformers import XLNetForSequenceClassification, XLNetTokenizer

# Load the AG News dataset
dataset = load_dataset('ag_news', split='train')

# Preprocess the dataset
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=512)

processed_dataset = dataset.map(preprocess_function, batched=True)

# Create a PyTorch Dataset
class TextClassificationDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TextClassificationDataset(processed_dataset['input_ids'], processed_dataset['label'])

# Create a DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Define the model and optimizer
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=4)
optimizer = transformers.AdamW(model.parameters(), lr=1e-5)

# Define the training loop
num_epochs = 3
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()

    model.eval()
    # Evaluate the model on a validation set here

# Save the fine-tuned model
model.save_pretrained('finetuned_xlnet')

In [ ]:
#resumen T5
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Define el modelo y tokenizer
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Texto de ejemplo
text = """
The new trailer for Deadpool & Wolverine may have provided a first look at the Marvel Cinematic Universe debut of a key X-Men villain.

Released during the Super Bowl on Sunday, the third Deadpool movie trailer sets up the long-awaited team-up between Ryan Reynolds' Deadpool and Hugh Jackman's Wolverine. The film also makes use of the MCU's multiverse, which includes bringing in other names from the X-Men world. One particular shot in the trailer reveals the back of a mystery character, who's shown to be bald and wearing a jacket with the collar pulled up. That just so happens to be the look of Cassandra Nova, an X-Men supervillain. The character is a dark shadow of Charles Xavier, having created herself by copying Professor X's DNA to be his de facto twin sister.

Beyond this quick shot in the trailer, there's no confirmation that this character is indeed Cassandra Nova. If so, however, the supervillain is most likely being portrayed by Emma Corrin. It's been known that Corrin had been cast to play a key villain role in the movie, but the part has ever since been kept under lock and key. At one point, they were rumored to play Danger, but that was debunked. Fans will have to wait for further confirmation to be sure, but the subtle tease may be a sign that the movie's primary antagonist will be revealed to be Corrin's Cassandra Nova.

There will be established X-Men characters from previous movies appearing in the film as well. That obviously includes Hugh Jackman's Wolverine, who's one of the main characters alongside Ryan Reynolds' Deadpool. The new trailer also revealed a first look at the return of Aaron Stanford's Pyro, marking the first time that incarnation of the character had been seen since X-Men: The Last Stand in 2006. These are just the appearances that were teased in the trailer, and chances are, there will be other major X-Men surprises in store for fans when they watch the movie.
"""

# Tokeniza el texto
inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=1024)

# Genera el resumen
outputs = model.generate(
    inputs['input_ids'],
    max_length=128,  # Longitud máxima del resumen
    num_beams=2,  # Número de diferentes resúmenes a generar
    early_stopping=True,  # Detiene la generación cuando encuentra una buena secuencia
)

# Decodifica y muestra el resumen
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Resumen: {summary}")

In [ ]:
#resumen gtp-2
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "gpt2-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

input_text = """
Marvel Studios está de vuelta. Aprovechando la celebración de la Super Bowl, uno de los más grandes eventos televisivos del año en Estados Unidos, la filial de Disney ha compartido el primer y extenso tráiler de su gran apuesta del 2024: Deadpool 3 o, como pasará a llamarse a partir de ahora, Deadpool & Wolverine.

Han sido dos minutos y medio de vídeo, bastantes teniendo en cuenta que el film acabó su rodaje el pasado 24 de enero, si bien este empezó en mayo del 2023. En el metraje podemos ver de nuevo al mercenario bocazas en acción, más dándole al palique que combatiendo, pero esta vez el protagonismo quizás se lo ha robado tener en pantalla, o casi, una vez más a Hugh Jackman como Wolverine.

Deadpool & Wolverine se estrenará en cines este 26 de julio. Tal y como hemos podido ver en este tráiler lo hará entregando a la audiencia un nuevo crossover, esperándose muchas sorpresas que, por supuesto, desde Marvel Studios se lo han querido guardar para una ocasión posterior, quizás para el mismo film, y aquí solo nos deja un adelanto bajo este sugerente y ¿apocalíptico? lema: "Everyone deserves a happy ending" (Todos merecen un final feliz en castellano).

A buen seguro los más fieles seguidores del MCU y Marvel encontrarán multitud de detalles y pistas, pero de momento lo más destacado de avance, aparte de reencontrarnos con sus protagonista, es la introducción del personaje en el Universo Cinematográfico de Marvel con un cierto rol destacado de la Autoridad de Variación Temporal (TVA por sus siglas en inglés) vista en la serie de Loki. Al parecer, este organismo busca reclutar a Wade Wilson, algo que el "héroe" parece aceptar con buen grado: "Tu pequeño universo cinematográfico va a cambiar para siempre. Soy el Mesías. Soy... el Jesús de Marvel", llega a decir el bocazas.

Deadpool & Wolverine está dirigida por Shawn Levy con Ryan Reynolds en el papel protagonista, esta vez acompañado por Hugh Jackman así como por Emma Corrin (The Crown), Matthew Macfadyen (Succession) así por veteranos de las dos primeras entregas de esta saga: Rob Delaney (Sugar Bear), Karan Soni (Dopinder), Leslie Uggams (Blind Al), Morena Baccarin (Vanessa ) y Stefan Kapicic (Coloso).
"""

# Set the padding token to the EOS token
tokenizer.pad_token = tokenizer.eos_token

inputs = tokenizer(input_text, return_tensors="pt", padding=False, truncation=False)
pad_token_id = tokenizer.pad_token_id

# Generate the summary
outputs = model.generate(inputs['input_ids'], max_length=20, num_beams=1, early_stopping=True, pad_token_id=pad_token_id)
summary_text = tokenizer.decode(outputs[0])

# Print the summary
print(summary_text)

In [ ]:
#Resumen usando XLNet 

from transformers import XLNetForSequenceClassification, XLNetTokenizer
from torch.nn import Softmax
import torch


# Define the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Carga el modelo XLNet y el tokenizador
model_name = "finetuned_xlnet"
model = XLNetForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)
tokenizer = XLNetTokenizer.from_pretrained(model_name)

# Ajusta el modelo para usar una función de activación Softmax en la última capa
model.classifier = Softmax(dim=1)

# Texto de entrada largo
input_text = """
In the past, humans were limited in their ability to communicate over long distances. This all changed with the invention of the telegraph by Samuel Morse in 1837. The telegraph revolutionized communication, enabling people to send messages almost instantly across long distances via electrical signals. However, the telegraph had some limitations, such as the need for trained operators to encode and decode messages, and the inability to communicate during electrical storms or when the telegraph wires were down.

The next major development in communication technology was the telephone, invented by Alexander Graham Bell in 1876. The telephone allowed for audio communication between two people in real time, and it quickly became a popular method of communication. However, the telephone also had its limitations, such as the need for a physical connection between the two parties and the inability to communicate with multiple people at once.

In the mid-20th century, communication technology took another leap forward with the invention of the fax machine. The fax machine allowed for the transmission of documents over phone lines, making it possible to send and receive written communications quickly and easily. However, the fax machine had its own set of limitations, such as the need for a dedicated phone line and the inability to transmit color images.

The most recent major development in communication technology has been the internet. The internet has revolutionized the way we communicate, enabling us to send and receive messages, documents, and multimedia content in real time, regardless of location. The internet has also made it possible to communicate with multiple people at once, facilitating collaboration and the sharing of ideas on a global scale.

Despite the many advances in communication technology, there are still challenges to be addressed. For example, while the internet has made it easier than ever to communicate, it has also given rise to new forms of misinformation and cybercrime. Additionally, the digital divide between those who have access to high-speed internet and those who do not can exacerbate existing social and economic inequalities.

In conclusion, communication technology has come a long way since the invention of the telegraph. From the telephone to the fax machine to the internet, each new development has brought with it new possibilities and challenges. As we continue to innovate and improve communication technology, it is important to consider both the benefits and the potential drawbacks, and to work towards solutions that promote equitable access and responsible use.
"""

# Tokeniza el texto de entrada
inputs = tokenizer(input_text, return_tensors="pt")

# Define the dataset and dataloader for fine-tuning
dataset = torch.utils.data.TensorDataset(inputs["input_ids"], inputs["attention_mask"])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)

# Define the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Fine-tune the model
num_epochs = 10
for epoch in range(num_epochs):
    for batch in dataloader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        probs = outputs[0]

        # Compute the loss
        labels = torch.zeros(probs.shape, device=device)
        labels[:, torch.argmax(probs)] = 1
        loss = loss_fn(probs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()

        # Update weights
        optimizer.step()

# Move input tensors to the same device as the model
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generate the summary
with torch.no_grad():
    outputs = model(**inputs)
    probs = outputs[0]
    top_probs, top_indexes = torch.topk(probs, 1, dim=1)
    summary_indexes = torch.where(top_indexes == 1)[1]
    summary_tokens = inputs["input_ids"][0][summary_indexes]
    summary_text = tokenizer.decode(summary_tokens)

# Imprime el resumen
print(summary_text)



In [ ]:
#Texto y traduccion con Bard
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


def traduccion(resumen):
    translator = Translator()
    whisper_language = translator.detect(resumen).lang
    translated_text = translator.translate(resumen, src=whisper_language, dest=target_language_code).text
    translated_text = titulo_texto + translated_text 
    print("Translated text:", translated_text)
    return translated_text
def auido_generation(resumen_esp):
    confirmation = input("You must confirm the licensing terms. Do you agree? (y/n): ")
    if confirmation.lower() != 'y':
        print("Exiting. License agreement not accepted.")
        exit()

    # Crear instancia de TTS
    tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to("cuda")

    # Generar audio
    tts.tts_to_file(
        resumen_esp,
        speaker_wav=rutaAudioIngles,
        file_path=rutaAudioGenerado,
        language=target_language_code
    )

target_language = "Spanish"
# Mapping between full names and ISO 639-1 codes
language_mapping = {'English': 'en', 'Spanish': 'es', 'Chinese (Simplified)': 'zh-cn', 'Japanese': 'ja'}
target_language_code = language_mapping[target_language]

with open("C://Users//R20//Desktop//videostraduccidos//resumen.txt", "r", encoding="utf-8") as archivo:
    texto = archivo.read()


# Modelo pre-entrenado
model_name = "facebook/bart-base"

# Cargar el modelo y el tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Preprocesamiento
encoded_text = tokenizer(texto, return_tensors="pt")

# Generar resumen
generated_tokens = model.generate(**encoded_text, max_new_tokens=150) 

# Decodificar resumen
resumen = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

# Imprimir el resumen
print(resumen)


new_text_trans = traduccion(resumen)
auido_generation(new_text_trans)
  

            

In [5]:
#texto scraping chatgpt

def auido_generation(resumen_esp):
    confirmation = input("You must confirm the licensing terms. Do you agree? (y/n): ")
    if confirmation.lower() != 'y':
        print("Exiting. License agreement not accepted.")
        exit()

    # Crear instancia de TTS
    tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to("cuda")

    # Generar audio
    tts.tts_to_file(
        resumen_esp,
        speaker_wav=rutaAudioIngles,
        file_path=rutaAudioGenerado,
        language=target_language_code
    )

target_language = "Spanish"
# Mapping between full names and ISO 639-1 codes
language_mapping = {'English': 'en', 'Spanish': 'es', 'Chinese (Simplified)': 'zh-cn', 'Japanese': 'ja'}
target_language_code = language_mapping[target_language]

with open("C://Users//R20//Desktop//videostraduccidos//resumen.txt", "r", encoding="utf-8") as archivo:
    new_text_trans = archivo.read()
    new_text_trans = titulo_texto + new_text_trans
auido_generation(new_text_trans)

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts
 > Text splitted to sentences.
['Los Zombies de Modern Warfare 3 causan enojos El Nuevo bundle incluye el regreso de los zombies a la franquicia, una característica altamente solicitada por los fanáticos.', 'Sin embargo, muchos jugadores consideran que el precio del bundle es excesivamente alto, especialmente teniendo en cuenta que el juego original fue lanzado hace casi una década.', 'Los jugadores expresan su frustración en las redes sociales, criticando el precio de $20 por un contenido que consideran no justifica ese coste.', 'Además, algunos señalan que los juegos más recientes de la serie ofrecen contenido similar o superior a precios más bajos.', 'La discusión sobre el valor del contenido descargable en los juegos antiguos sigue siendo un tema candente en la comunidad de juegos, con opiniones divididas entre aquellos que creen que los precios son injustificados y aquellos dispuestos a pa

In [ ]:
#Texto y traduccion

target_language = "Spanish"
# Mapping between full names and ISO 639-1 codes
language_mapping = {'English': 'en', 'Spanish': 'es', 'Chinese (Simplified)': 'zh-cn', 'Japanese': 'ja'}
target_language_code = language_mapping[target_language]

def summarize(filepath, language="english", sentences_count=5):
    try:
        with open(filepath, "r") as f:
            text = f.read()
    except FileNotFoundError:
        print(f"Error: Archivo no encontrado: {filepath}")
        return None

    parser = PlaintextParser.from_string(text, Tokenizer(language))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, sentences_count)
    return ' '.join([str(sentence) for sentence in summary])

def traduccion(resumen):
    translator = Translator()
    whisper_language = translator.detect(resumen).lang
    translated_text = translator.translate(resumen, src=whisper_language, dest=target_language_code).text
    translated_text = titulo_texto + translated_text 
    print("Translated text:", translated_text)
    return translated_text

def auido_generation(resumen_esp):
    confirmation = input("You must confirm the licensing terms. Do you agree? (y/n): ")
    if confirmation.lower() != 'y':
        print("Exiting. License agreement not accepted.")
        exit()

    # Crear instancia de TTS
    tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to("cuda")

    # Generar audio
    tts.tts_to_file(
        resumen_esp,
        speaker_wav=rutaAudioIngles,
        file_path=rutaAudioGenerado,
        language=target_language_code
    )

def audio_speed():
    cambio_velocidad = 1
    velocidad_correcta = False
    y_generated, sr_generated = librosa.load(rutaAudioGenerado)
    duration_original_esp = len(y_generated) / sr_generated
    print(f"La duración del audio original es de {duration_original_esp:.2f} segundos")
    sr_adjusted = int(sr_generated * cambio_velocidad) 
    y_adjusted = librosa.effects.time_stretch(y_generated, rate=cambio_velocidad)
    duration_adjusted = len(y_adjusted) / sr_adjusted

    sf.write(rutaAudioGeneradoNuevaVelocidad, y_adjusted, sr_generated)
  

    


if __name__ == "__main__":
    filepath = "C://Users//R20//Desktop//videostraduccidos//resumen.txt"  # Reemplaza con la ruta real de tu archivo
    summary = summarize(filepath)
    if summary:
        print(summary)
        new_text_trans = traduccion(summary)
        if new_text_trans:
            auido_generation(new_text_trans)
    
    


In [6]:
#Cambiar Velocidad del audio
# Cargar un archivo de audio
input_audio_file = rutaAudioGenerado

stretch_factor =1.5  # Por ejemplo, para aumentar la duración en un 50%

# Aplicar el estiramiento del tiempo
input_audio_data, sr = sf.read(input_audio_file)
output_audio_data = pyrubberband.time_stretch(input_audio_data, sr, rate=stretch_factor)

# Guardar el archivo de audio resultante
sf.write(rutaAudioGeneradoNuevaVelocidad, output_audio_data, sr)

audio_widget = Audio(filename=rutaAudioGeneradoNuevaVelocidad, autoplay=False)
display(audio_widget)


In [ ]:
#Crear video con imagenes

audio_path = rutaAudioGeneradoNuevaVelocidad
image_paths = []
for filename in os.listdir(ruta_imagenes):
    if filename.endswith((".jpg", ".png", ".PNG")):
        image_paths.append(os.path.join(ruta_imagenes, filename))

clips = []
y_generated, sr_generated = librosa.load(audio_path)
duration_original = len(y_generated) / sr_generated
n_segundos = duration_original/len(image_paths)
n_frames = 30
iteraciones_for = 2
for image_path in image_paths:
    iteraciones_for +=1
    # Cargar la imagen
    image = Image.open(image_path)

    # Estirar la imagen (ejemplo: duplicar su ancho)
    image = image.resize((image.width , image.height ))

    # Convertir a formato compatible con OpenCV
    image_opencv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGRA)

    # Crear la animación
    alto, ancho, _ = image_opencv.shape
    if iteraciones_for % 2 == 0:
        matriz_transformacion = np.array([[2, 0, -image.width], [0, 1, 1]], dtype=np.float32)
    else:
        matriz_transformacion = np.array([[2, 0, 1], [0, 1, 1]], dtype=np.float32)
    frames = []
    pixeles_por_segundo = image.width / n_segundos
    pixeles_por_frame = pixeles_por_segundo/n_frames
    
    for i in range(int(n_segundos*n_frames)):
        imagen_transformada = cv2.warpAffine(image_opencv, matriz_transformacion, (ancho, alto))
        # Convertir de BGR a RGB para mantener los colores originales
        imagen_transformada = cv2.cvtColor(imagen_transformada, cv2.COLOR_BGRA2RGB)
        frames.append(imagen_transformada)
        if iteraciones_for % 2 == 0:
            matriz_transformacion[0, 2] += pixeles_por_frame
        else:
            matriz_transformacion[0, 2] -= pixeles_por_frame

    # Crear el clip de video con la animación
    clip = mpe.ImageSequenceClip(frames, fps=30).set_duration(duration_original/len(image_paths))
    clips.append(clip)

video = mpe.concatenate_videoclips(clips)
video = video.set_audio(mpe.AudioFileClip(audio_path))

video.fps = 30  # Ajusta el frame rate

video = video.resize((1080, 1920))  # Dimensiones de TikTok

video_path = ruta_video_final
video.write_videofile(video_path)

In [ ]:
#Generar letreros en el video.
video = mp.VideoFileClip(rutaVideo)

texto_que_cubre = "Jugadores de Palworld \n piden mejor construccion"
fontsize = 40
duration = video.duration

posicion = ("center", 220)
subtitle = TextClip(texto_que_cubre, fontsize=fontsize,font="Bro Alex.ttf", color='white', bg_color='black')
subtitle_clip = subtitle.set_pos(posicion).set_duration(duration)
#subtitle_clips.append(subtitle_clip)


# Overlay the subtitle clips on the video clip
video_with_subtitles = CompositeVideoClip([video, subtitle_clip])

# Save the resulting clip
video_with_subtitles.write_videofile(ruta_video_final)

In [ ]:
#generar subtitulos y poner imagenes que coincidan

video = mp.VideoFileClip(rutaVideo)

def poner_imagenes(nombre_imagen, inicio):
    #Cargar la imagen y redimensionarla
    imagen = (mpe.ImageClip(ruta_imagenes + "conocidas//"+nombre_imagen+".png")
            .resize((400, 300))
            .set_duration(1)
            .set_start(inicio)
            )  # Establecer la duración de la imagen a 3 segundos

    ancho_video, alto_video = video.size

    # Calcular las coordenadas del centro
    x_centro = ancho_video / 2 - imagen.w / 2
    y_centro = alto_video / 2 - imagen.h / 2

    superposicion = imagen.set_pos((x_centro, y_centro))
    return superposicion


palabras_clave = ["STEAM","XBOX","NINTENDO"]  # Ajusta según tus necesidades
whisper_text = new_text_trans.replace('.', '. ').replace('?', '? ')



words = whisper_text.split()

# Group the words into pairs
#subtitle_lines = [' '.join(words[i:i+2]) for i in range(0, len(words), 2)]
subtitle_lines = []
i = 0
while i < len(words):
    if i + 1 < len(words) and words[i+1].strip() != '':
        # Si el siguiente elemento de la lista no está vacío, unir las dos palabras con un espacio
        subtitle_lines.append(' '.join([words[i], words[i+1]]))
        i += 2
    else:
        # Si el siguiente elemento de la lista está vacío o es el último elemento, agregar solo la palabra actual
        subtitle_lines.append(words[i])
        i += 1
''' ota forma de cortar
for i in range(0, len(words), 2):
    if len(words[i]) > 8:
        subtitle_lines.append(words[i])
        subtitle_lines.append(words[i+1])
    else:
        subtitle_lines.append(' '.join([words[i], words[i+1]]))'''

# Remove any empty strings from the list of subtitle lines
subtitle_lines = [line.strip() for line in subtitle_lines if line.strip()]

# Create a list to store the text clips
subtitle_clips = []

# Set the font size and duration for each subtitle line
fontsize = 85
# Register the font
#font = ImageFont.truetype("Bubblegum.ttf", fontsize)
duration = video.duration / len(subtitle_lines)

# Initialize a variable to keep track of the current line number
line_number = 0

# Subtitle Positioning:
subtitle_position = ("center", 1250)  # Example position adjustment
subtitle_position_shadow = ("center", 1238)
# Create a TextClip for each subtitle line and add it to the list
for i, line in enumerate(subtitle_lines):
    line = line.upper()
    if line_number % 3 == 0:
        color = 'red'
    elif line_number % 3 == 1:
        color = 'yellow'
    else:
        color = 'white'
    subtitle = TextClip(line, fontsize=fontsize,font="ProtestStrike-Regular.ttf", color= color)
    subtitle_shadow = TextClip(line, fontsize=fontsize+3.9,font="ProtestStrike-Regular.ttf", color= "black")
    if any(palabra in line for palabra in palabras_clave):
        # Encontrar la primera palabra clave en la línea
        for palabra in palabras_clave:
            if palabra in line:
                palabra_clave_encontrada = palabra
                break

        display(palabra_clave_encontrada)
        
        subtitle_clip = subtitle_shadow.set_duration(duration).set_pos(subtitle_position_shadow).set_start(i * duration)
        subtitle_clips.append(subtitle_clip)
        subtitle_clip = subtitle.set_duration(duration).set_pos(subtitle_position).set_start(i * duration)
        subtitle_clips.append(subtitle_clip)
        superposicion = poner_imagenes(palabra_clave_encontrada, i * duration)
        subtitle_clips.append(superposicion)
    else:
        subtitle_clip = subtitle_shadow.set_duration(duration).set_pos(subtitle_position_shadow).set_start(i * duration)
        subtitle_clips.append(subtitle_clip)
        subtitle_clip = subtitle.set_duration(duration).set_pos(subtitle_position).set_start(i * duration)
        subtitle_clips.append(subtitle_clip)

    # Increment the line number
    line_number += 1

# Overlay the subtitle clips on the video clip
video_with_subtitles = CompositeVideoClip([video] + subtitle_clips)

# Save the resulting clip
video_with_subtitles = video_with_subtitles.set_duration(5)  # Establecer la duración antes de escribir el archivo
video_with_subtitles.write_videofile(ruta_video_final)

In [ ]:
#Poner imagen en el video


video = mpe.VideoFileClip(rutaVideo)

 #Cargar la imagen y redimensionarla
#imagen = (mpe.ImageClip(ruta_imagenes + "conocidas//patito.gif")
imagen = (mpe.VideoFileClip(ruta_imagenes + "conocidas//gatomostacho1.gif", has_mask=True)         
           .resize((800, 600))
           .set_duration(5)
            )  # Establecer la duración de la imagen a 3 segundos
imagen_final = (mpe.VideoFileClip(ruta_imagenes + "conocidas//gatomostacho2.gif", has_mask=True)         
           .resize((800, 600))
           .set_duration(4)
            )
imagen_medio = (mpe.VideoFileClip(ruta_imagenes + "conocidas//gatomostacho3.gif", has_mask=True)         
           .resize((800, 600))
           .set_duration(4)
            )

# Crear clip del GIF al final
clip_final = imagen_final.set_start(video.duration - 5)
clip_medio = imagen_medio.set_start(video.duration//2 - 1)

ancho_video, alto_video = video.size
gifs_clips = []
# Calcular las coordenadas del centro
x_centro = ancho_video / 2 - imagen.w / 2
y_centro = alto_video / 2 - imagen.h / 2

superposicion = imagen.set_pos((-250, 750))
superposicion2 = clip_final.set_pos((500, 750))
superposicion3 = clip_medio.set_pos((100, 750))

gifs_clips.append(superposicion)
gifs_clips.append(superposicion2)
gifs_clips.append(superposicion3)
video_con_superposicion = CompositeVideoClip([video]+ gifs_clips)


video_con_superposicion.write_videofile(ruta_video_final)

In [7]:
#video imagenes, letrero, subtitulo, gif.

#Crear video con imagenes

audio_path = rutaAudioGeneradoNuevaVelocidad
image_paths = []
for filename in os.listdir(ruta_imagenes):
    if filename.endswith((".jpg", ".png", ".PNG")):
        image_paths.append(os.path.join(ruta_imagenes, filename))

clips = []
y_generated, sr_generated = librosa.load(audio_path)
duration_original = len(y_generated) / sr_generated
n_segundos = duration_original/len(image_paths)
n_frames = 30
iteraciones_for = 2
for image_path in image_paths:
    iteraciones_for +=1
    # Cargar la imagen
    image = Image.open(image_path)

    # Estirar la imagen (ejemplo: duplicar su ancho)
    image = image.resize((image.width , image.height ))

    # Convertir a formato compatible con OpenCV
    image_opencv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGRA)

    # Crear la animación
    alto, ancho, _ = image_opencv.shape
    if iteraciones_for % 2 == 0:
        matriz_transformacion = np.array([[2, 0, -image.width], [0, 1, 1]], dtype=np.float32)
    else:
        matriz_transformacion = np.array([[2, 0, 1], [0, 1, 1]], dtype=np.float32)
    frames = []
    pixeles_por_segundo = image.width / n_segundos
    pixeles_por_frame = pixeles_por_segundo/n_frames
    
    for i in range(int(n_segundos*n_frames)):
        imagen_transformada = cv2.warpAffine(image_opencv, matriz_transformacion, (ancho, alto))
        # Convertir de BGR a RGB para mantener los colores originales
        imagen_transformada = cv2.cvtColor(imagen_transformada, cv2.COLOR_BGRA2RGB)
        frames.append(imagen_transformada)
        if iteraciones_for % 2 == 0:
            matriz_transformacion[0, 2] += pixeles_por_frame
        else:
            matriz_transformacion[0, 2] -= pixeles_por_frame

    # Crear el clip de video con la animación
    clip = mpe.ImageSequenceClip(frames, fps=30).set_duration(duration_original/len(image_paths))
    clips.append(clip)

video = mpe.concatenate_videoclips(clips)
video = video.set_audio(mpe.AudioFileClip(audio_path))

video.fps = 30  # Ajusta el frame rate

video = video.resize((1080, 1920))  # Dimensiones de TikTok

video_path = ruta_video_final
video.write_videofile(video_path)



#Generar letreros en el video.
video = mp.VideoFileClip(ruta_video_final)

texto_que_cubre = titulo
fontsize = 50
duration = video.duration

posicion = ("center", 220)
subtitle = TextClip(texto_que_cubre, fontsize=fontsize,font="Bro Alex.ttf", color='white', bg_color='black')

subtitle_clip = subtitle.set_pos(posicion).set_duration(duration)
#subtitle_clips.append(subtitle_clip)

gifs_clips = []

gifs_clips.append(subtitle_clip)

#imagen = (mpe.ImageClip(ruta_imagenes + "conocidas//patito.gif")
imagen = (mpe.VideoFileClip(ruta_imagenes + "conocidas//gatomostacho1.gif", has_mask=True)         
           .resize((800, 600))
           .set_duration(5)
            )  # Establecer la duración de la imagen a 3 segundos
imagen_final = (mpe.VideoFileClip(ruta_imagenes + "conocidas//gatomostacho2.gif", has_mask=True)         
           .resize((800, 600))
           .set_duration(5)
            )
imagen_medio = (mpe.VideoFileClip(ruta_imagenes + "conocidas//gatomostacho3.gif", has_mask=True)         
           .resize((800, 600))
           .set_duration(5)
            )

# Crear clip del GIF al final
clip_final = imagen_final.set_start(video.duration - 6)
clip_medio = imagen_medio.set_start(video.duration//2 - 1)

ancho_video, alto_video = video.size

# Calcular las coordenadas del centro
x_centro = ancho_video / 2 - imagen.w / 2
y_centro = alto_video / 2 - imagen.h / 2

superposicion = imagen.set_pos((-250, 750))
superposicion2 = clip_final.set_pos((500, 750))
superposicion3 = clip_medio.set_pos((100, 750))

gifs_clips.append(superposicion)
gifs_clips.append(superposicion2)
gifs_clips.append(superposicion3)

def poner_imagenes(nombre_imagen, inicio):
    #Cargar la imagen y redimensionarla
    imagen = (mpe.ImageClip(ruta_imagenes + "conocidas//"+nombre_imagen+".png")
            .resize((400, 300))
            .set_duration(1)
            .set_start(inicio)
            )  # Establecer la duración de la imagen a 3 segundos

    ancho_video, alto_video = video.size

    # Calcular las coordenadas del centro
    x_centro = ancho_video / 2 - imagen.w / 2
    y_centro = alto_video / 2 - imagen.h / 2

    superposicion = imagen.set_pos((x_centro, y_centro))
    return superposicion


palabras_clave = ["STEAM","XBOX","NINTENDO","MARVEL"]  # Ajusta según tus necesidades
whisper_text = new_text_trans.replace('.', '. ').replace('?', '? ')

words = whisper_text.split()

# Group the words into pairs
#subtitle_lines = [' '.join(words[i:i+2]) for i in range(0, len(words), 2)]
subtitle_lines = []
i = 0
while i < len(words):
    if i + 1 < len(words) and words[i+1].strip() != '':
        # Si el siguiente elemento de la lista no está vacío, unir las dos palabras con un espacio
        subtitle_lines.append(' '.join([words[i], words[i+1]]))
        i += 2
    else:
        # Si el siguiente elemento de la lista está vacío o es el último elemento, agregar solo la palabra actual
        subtitle_lines.append(words[i])
        i += 1

# Remove any empty strings from the list of subtitle lines
subtitle_lines = [line.strip() for line in subtitle_lines if line.strip()]

# Create a list to store the text clips
subtitle_clips = []

# Set the font size and duration for each subtitle line
fontsize = 85
# Register the font
#font = ImageFont.truetype("Bubblegum.ttf", fontsize)
duration = video.duration / len(subtitle_lines)

# Initialize a variable to keep track of the current line number
line_number = 0

# Subtitle Positioning:
subtitle_position = ('center', 1250)  # Example position adjustment
subtitle_position_shadow = ("center", 1238)
# Create a TextClip for each subtitle line and add it to the list
for i, line in enumerate(subtitle_lines):
    line = line.upper()
    if line_number % 3 == 0:
        color = '#FF0000'
    elif line_number % 3 == 1:
        color = '#FFF444'
    else:
        color = 'white'
    subtitle = TextClip(line, fontsize=fontsize,font="ProtestStrike-Regular.ttf", color=color, )
    
    subtitle_shadow = TextClip(line, fontsize=fontsize+3.9,font="ProtestStrike-Regular.ttf", color= "black")
    if any(palabra in line for palabra in palabras_clave):
        # Encontrar la primera palabra clave en la línea
        for palabra in palabras_clave:
            if palabra in line:
                palabra_clave_encontrada = palabra
                break

        display(palabra_clave_encontrada)
        
        subtitle_clip = subtitle_shadow.set_duration(duration).set_pos(subtitle_position_shadow).set_start(i * duration)
        gifs_clips.append(subtitle_clip)
        subtitle_clip = subtitle.set_duration(duration).set_pos(subtitle_position).set_start(i * duration)
        gifs_clips.append(subtitle_clip)
        superposicion = poner_imagenes(palabra_clave_encontrada, i * duration)
        gifs_clips.append(superposicion)
    else:
        subtitle_clip = subtitle_shadow.set_duration(duration).set_pos(subtitle_position_shadow).set_start(i * duration)
        gifs_clips.append(subtitle_clip)
        subtitle_clip = subtitle.set_duration(duration).set_pos(subtitle_position).set_start(i * duration)
        gifs_clips.append(subtitle_clip)

    # Increment the line number
    line_number += 1

# Overlay the subtitle clips on the video clip
video_con_superposicion = CompositeVideoClip([video]+ gifs_clips)
video_con_superposicion = video_con_superposicion.set_duration(video.duration)

video_con_superposicion.write_videofile(ruta_video_final_editado)

# Save the resulting clip
#video_with_subtitles = video_with_subtitles.set_duration(video.duration)  # Establecer la duración antes de escribir el archivo
#video_with_subtitles.write_videofile(ruta_video_final)

Moviepy - Building video C://Users//R20//Desktop//videostraduccidos//videos_creados//video15.mp4.
MoviePy - Writing audio in video15TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video C://Users//R20//Desktop//videostraduccidos//videos_creados//video15.mp4



Moviepy - Done !
Moviepy - video ready C://Users//R20//Desktop//videostraduccidos//videos_creados//video15.mp4
Moviepy - Building video C://Users//R20//Desktop//videostraduccidos//videos_creados//finalesvideo15.mp4.
MoviePy - Writing audio in finalesvideo15TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video C://Users//R20//Desktop//videostraduccidos//videos_creados//finalesvideo15.mp4



t: 100%|█████████▉| 1273/1274 [02:40<00:00, 17.87it/s, now=None]WARNING:py.warnings:c:\Python39\lib\site-packages\moviepy\video\io\ffmpeg_reader.py:123: UserWarning: Warning: in file C://Users//R20//Desktop//videostraduccidos//videos_creados//video15.mp4, 6220800 bytes wanted but 0 bytes read,at frame 1273/1274, at time 42.43/42.45 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready C://Users//R20//Desktop//videostraduccidos//videos_creados//finalesvideo15.mp4
